# **Modelo de Recomendação**
Nesse notebok será trabalhado no modelo de recomendação de produtos com dados de usuários de Olist, que foram selecionados e gerados a partir do notebook **analise_exploratoria_olist.ipnyb** (recomendo a leitura). Além de treinarmos o modelo para recomendação, vamos validar sua qualidade de previsão, testar dados absolutos e normalizados, e alguns modelos para entender qual seria a melhor opção para essa base de dados.

## **Resumo do Modelo**
AAAAAAAAAAAAAAA

In [43]:
import random
import numpy as np
import pandas as pd

#Pré-processamento
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

#Modelos
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

#Métricas de avaliação
from sklearn.metrics import (
    roc_auc_score, log_loss, f1_score, precision_score,
    recall_score, average_precision_score)


In [44]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,order_id,customer_id,customer_unique_id,customer_city,customer_state,product_id,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,sao paulo,SP,87285b34884572647811a353c7ac498a,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,4
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,barreiras,BA,595fac2a385ac33a80bd5114aec74eb8,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,4
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,vianopolis,GO,aa4383b373c6aca5d8797843e5594415,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,sao goncalo do amarante,RN,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,27.20,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,santo andre,SP,65266b2da20d04dbe00c5c2d3bb7859e,19.90,8.72,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,5


In [45]:
#Selecionando características de produtos e usuários
produto_features = ["price", "freight_value", "product_category_name", 
                    "product_name_lenght", "product_description_lenght", "product_photos_qty",
                    "product_weight_g", "product_length_cm", "product_height_cm",
                    "product_width_cm"]
cliente_features = ["customer_city", "customer_state"]

In [46]:
#Preenchendo o categoria que não são conhecidas com desconhecido
dataset["product_category_name"] = np.where(
    dataset["product_category_name"].isna(),
    "unknown",
    dataset["product_category_name"]
)

**Gerando uma amostra negativa**

No dataset existe apenas dados de compras realizadas, como a intenção é desenvolver um modelo de classificação para recomendar de produtos, será necessário criar uma amostra aleatória de compras não realizadas para auxiliar o modelo no entendimento dos dados.

- **Amostragem Negativa por Similaridade** \
A opção utilizada para amostragem é por similaridade, aonde gera dados negativos de produtos não comprados, mas com certa similaridade de produtos que esses clientes compraram. Aumentando a possibilidade de voltarem a comprar no ecommerce.

In [47]:
# Fazendo a lista de positivos para os produtos que foram comprados
positivos = dataset[["customer_unique_id", "product_id"]].drop_duplicates()
positivos['target'] = 1

clientes = dataset["customer_unique_id"].unique()
produtos = dataset["product_id"].unique()

negativos = []
#Variável que dá proporção de negativos para positivos
n_amostras = 1

#Realizando a amostra negativa por similaridade
for client_id in dataset['customer_unique_id'].unique():
    comprados = dataset[dataset['customer_unique_id'] == client_id]['product_category_name'].unique()
    produtos_similares = dataset[~dataset['product_category_name'].isin(comprados)]['product_id'].unique()

    for _ in range(n_amostras):
        p = random.choice(produtos_similares)
        negativos.append((client_id, p, 0))
    
negativos = pd.DataFrame(negativos, columns=['customer_unique_id', 'product_id', 'target'])

#Agrupando as interações positivas e negativas
interacoes = pd.concat([positivos, negativos])

#Agrupando o restante o df com as características
dados = interacoes.merge(dataset.drop_duplicates("customer_unique_id")[["customer_unique_id"] + cliente_features], 
                            on="customer_unique_id", how="left")
dados = dados.merge(dataset.drop_duplicates("product_id")[["product_id"] + produto_features], 
                          on="product_id", how="left")

**Pré-processamento**

| Variáveis               | Tratamento                                                                      |
| --------------------- | ---------------------------------------------------------------------------------------- |
| Numéricas com Poucos Valores Vazios | Aplicação da **média** nos dados vazios|
| Numéricas com Muitos Valores Vazios | Aplicação da **mediana** nos dados vazios|
| Numéricas | Aplicação da **normalização** das escalas|
| Categóricas | Aplicação de **dummies**|

In [48]:
X = dados[cliente_features + produto_features]
y = dados['target']

num_features = list(dados.select_dtypes("float").columns)
cat_features = list(dados.select_dtypes("object").columns)
cat_features.remove("customer_unique_id")
cat_features.remove("product_id")

In [49]:
dados_nulos = dados[num_features].isnull().sum()
poucos_nulos = dados_nulos[dados_nulos <= 50].index.tolist()
muitos_nulos = dados_nulos[dados_nulos > 50].index.tolist()

In [50]:
num_pipeline_poucos = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

num_pipeline_muitos = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num_poucos', num_pipeline_poucos, poucos_nulos),
    ('num_muitos', num_pipeline_muitos, muitos_nulos),
    ('cat', cat_pipeline, cat_features)
])

**Avaliação de Modelos**

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

modelos = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42)
}

In [58]:
# Avaliação de métricas
resultados = []
modelos_treinados = {}

for nome, classificador in modelos.items():
    pipeline = Pipeline(steps=[
        ('preprocess', preprocessor),
        ('classificador', classificador)
    ])
    
    pipeline.fit(X_train, y_train)
    modelos_treinados[nome] = pipeline
    
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1]

    resultados.append({
        "Modelo": nome,
        "AUC-ROC": roc_auc_score(y_test, y_proba),
        "LogLoss": log_loss(y_test, y_proba),
        "F1-Score": f1_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "Average Precision": average_precision_score(y_test, y_proba)
    })

df_resultados = pd.DataFrame(resultados).set_index("Modelo")

C:\Users\Computador\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning: [22:38:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 69229, number of negative: 67442
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3025
[LightGBM] [Info] Number of data points in the train set: 136671, number of used features: 823
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506538 -> initscore=0.026152
[LightGBM] [Info] Start training from score 0.026152


| Métrica               | Interpretação breve                                                                      | Faixa ideal            |
| --------------------- | ---------------------------------------------------------------------------------------- | ---------------------- |
| **AUC-ROC**           | Mede quão bem o modelo separa positivos de negativos (0.5 = aleatório, 1.0 = perfeito)   | > 0.7 bom, > 0.8 ótimo |
| **LogLoss**           | Penaliza previsões erradas com alta confiança (quanto **menor**, melhor)                 | < 0.6 ideal            |
| **F1-Score**          | Equilíbrio entre precisão e recall – importante quando há desbalanceamento               | > 0.6 razoável         |
| **Precision**         | Entre os que o modelo disse que comprariam, quantos realmente compraram?                 | > 0.6 é bom            |
| **Recall**            | Entre os que realmente compraram, quantos o modelo acertou?                              | > 0.6 é bom            |
| **Average Precision** | Área sob curva Precision x Recall, útil para listas ranqueadas (ex: top-N recomendações) | > 0.7 ideal            |


In [53]:
df_resultados

,AUC-ROC,LogLoss,F1-Score,Precision,Recall,Average Precision
Modelo,,,,,,
RandomForest,0.701796,0.649723,0.640039,0.682282,0.602721,0.748785
XGBoost,0.718397,0.609830,0.642172,0.678783,0.609308,0.750327
LightGBM,0.701869,0.627845,0.617522,0.681597,0.564458,0.736580
CatBoost,0.724814,0.607961,0.650163,0.681721,0.621398,0.754333


# **Recomendação**

In [54]:
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",", usecols=['customer_id', 'order_purchase_timestamp'])
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",", usecols=['customer_id', 'customer_unique_id'])

data_referencia = pd.to_datetime("2018-10-18")
olist_orders["order_purchase_timestamp"] = pd.to_datetime(olist_orders["order_purchase_timestamp"], format='%Y-%m-%d %H:%M:%S')
olist_orders['dias_desde_da_ultima_compra'] = (data_referencia - olist_orders["order_purchase_timestamp"]).dt.days.clip(lower=0)

customer_id = olist_orders[(olist_orders['dias_desde_da_ultima_compra'] >= 30) &(olist_orders['dias_desde_da_ultima_compra'] <= 90)]
customer_id = customer_id["customer_id"].unique()

olist_customers = olist_customers[olist_customers["customer_id"].isin(customer_id)]
customer_unique_id = olist_customers["customer_unique_id"].unique()

In [56]:
def gerar_recomendacoes_para_clientes(dados, modelo, customer_ids, cliente_features, produto_features, top_n=5):
    todos_produtos = dados['product_id'].unique()
    recomendacoes_gerais = []

    for cliente_id in customer_ids:
        # Produtos já comprados por esse cliente
        produtos_comprados = dados[dados['customer_unique_id'] == cliente_id]['product_id'].unique()
        produtos_nao_comprados = [p for p in todos_produtos if p not in produtos_comprados]

        if len(produtos_nao_comprados) == 0:
            continue  # pula cliente que já comprou tudo

        # Informações do cliente
        try:
            cliente_info = dados[dados['customer_unique_id'] == cliente_id].iloc[0]
        except IndexError:
            continue  # cliente não encontrado nos dados

        # Criar linhas combinando cliente com produtos não comprados
        recomendacoes = []
        for produto_id in produtos_nao_comprados:
            prod_linha = dados[dados['product_id'] == produto_id]
            if prod_linha.empty:
                continue  # produto inexistente nos dados

            prod_info = prod_linha.iloc[0]

            linha = {
                'customer_unique_id': cliente_id,
                'customer_state': cliente_info['customer_state'],
                'customer_city': cliente_info['customer_city'],
                'product_id': produto_id,
                'product_category_name': prod_info['product_category_name'],
                'price': prod_info['price'],
                'freight_value': prod_info['freight_value'],
                'product_name_lenght': prod_info['product_name_lenght'],
                'product_description_lenght': prod_info['product_description_lenght'],
                'product_photos_qty': prod_info['product_photos_qty'],
                'product_weight_g': prod_info['product_weight_g'],
                'product_length_cm': prod_info['product_length_cm'],
                'product_height_cm': prod_info['product_height_cm'],
                'product_width_cm': prod_info['product_width_cm']
            }

            recomendacoes.append(linha)

        if not recomendacoes:
            continue

        df_recomendacoes = pd.DataFrame(recomendacoes)

        # Previsões
        X_recomendacao = df_recomendacoes[cliente_features + produto_features]
        scores = modelo.predict_proba(X_recomendacao)[:, 1]
        df_recomendacoes['score'] = scores

        # Top N por cliente
        top_recs = df_recomendacoes.sort_values(by='score', ascending=False).head(top_n)
        recomendacoes_gerais.append(top_recs)

    # Concatenar todas as recomendações
    resultado_final = pd.concat(recomendacoes_gerais, ignore_index=True)
    return resultado_final[['customer_unique_id', 'product_id', 'product_category_name', 'score']]


In [ ]:
customer_ids = dados[dados["customer_unique_id"].isin(customer_unique_id)]
customer_ids = customer_ids["customer_unique_id"].unique()
modelo_escolhido = modelos_treinados["CatBoost"]

top_recomendacoes = gerar_recomendacoes_para_clientes(
    dados=dados,
    modelo=modelo_escolhido,
    customer_ids=customer_ids,
    cliente_features=cliente_features,
    produto_features=produto_features,
    top_n=5
)

print(top_recomendacoes)